In [28]:
import psycopg2
import pandas as pd

# Credenciales de la base de datos
dbname = "DBInmerso"
user = "postgres"
password = "postgres"
host = "localhost"  # Cambia esto según tu configuración
port = "5432"

# Crear la cadena de conexión
connection_string = f"dbname={dbname} user={user} password={password} host={host} port={port}"

# Intentar establecer la conexión
try:
    connection = psycopg2.connect(connection_string)

    # Consulta para concatenar las tablas
    query_concatenar = """
        SELECT
            u.dni,
            u.nombre,
            u.apellido,
            u.edad,
            u.fecha_de_nacimiento,
            u.id_solicitud,
            u.usuario_solicitante,
            u.oficio_especial,
            r.ingresos,
            d.grado_dis,
            fn.tipo_fam_num,
            p.valoracion_patrimonio,
            p.coche,
            p.arrendador
        FROM
            usuarios u
        JOIN renta r ON u.dni = r.dni
        JOIN discapacidad d ON u.dni = d.dni
        JOIN familia_numerosa fn ON u.dni = fn.dni
        JOIN patrimonio p ON u.dni = p.dni;
    """

    # Crear un DataFrame con los resultados de la consulta concatenada
    df = pd.read_sql_query(query_concatenar, connection)

except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")

finally:
    # Cerrar la conexión en el bloque 'finally' para asegurar que se cierre
    if connection:
        connection.close()


#Ponderaciones
edad_weight = 0.1
discapacidad_weight = 0.1
familia_weight = 0.1
renta_wheigt = 0.1
coche_wight = 0.1
oficio_especial_wight = 0.1
arrendador_wight = 0.1

df

C:\Users\DELL\AppData\Local\Temp\ipykernel_3204\3735356183.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_concatenar, connection)


,dni,nombre,apellido,edad,fecha_de_nacimiento,id_solicitud,usuario_solicitante,oficio_especial,ingresos,grado_dis,tipo_fam_num,valoracion_patrimonio,coche,arrendador
0,95822412,Joan,Nuñez,62,1956-06-23,52,False,True,2336.0,2,1,2000000.0,True,True
1,46913810,Jerónimo,Fabra,70,1942-01-20,458,False,True,4247.0,2,2,441843.0,False,False
2,28728463,Nacio,Bravo,61,1955-02-11,1386,False,True,3132.0,0,0,138429.0,False,False
3,83197857,Ascensión,Durán,60,1959-08-03,1210,True,False,3452.0,1,0,347455.0,False,False
4,66629388,Cristina,Iñiguez,57,1908-09-15,62,False,False,1517.0,0,2,409487.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,69148126,Isa,Pozo,57,1911-08-23,130,True,False,3718.0,2,0,47721.0,False,True
1496,45113084,Gabriel,Castejón,75,1922-06-15,26,True,True,2269.0,1,1,318972.0,False,True
1497,80439867,Luisina,Olivera,69,1949-10-25,1372,True,True,4993.0,0,2,293549.0,False,False
1498,55405943,Javi,Corbacho,88,1914-11-30,1072,True,True,1551.0,0,2,119297.0,False,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dni                    1500 non-null   object 
 1   nombre                 1500 non-null   object 
 2   apellido               1500 non-null   object 
 3   edad                   1500 non-null   int64  
 4   fecha_de_nacimiento    1500 non-null   object 
 5   id_solicitud           1500 non-null   int64  
 6   usuario_solicitante    1500 non-null   bool   
 7   oficio_especial        1500 non-null   bool   
 8   ingresos               1500 non-null   float64
 9   grado_dis              1500 non-null   int64  
 10  tipo_fam_num           1500 non-null   int64  
 11  valoracion_patrimonio  1500 non-null   float64
 12  coche                  1500 non-null   bool   
 13  arrendador             1500 non-null   bool   
dtypes: bool(4), float64(2), int64(4), object(4)
memory usage

### EDAD.SCORE

In [11]:
def puntaje_edad_nueva(edad):
    if edad < 60:
        return 1
    elif 60 <= edad < 70:
        return 20
    elif 70 <= edad < 80:
        return 30
    else:  # Edad >= 80
        return 40

def procesar_edad(df):
    # Copiar el DataFrame para evitar modificar el original
    df_edad = df.copy()

    # Aplicar la función a la columna 'edad' y crear la nueva columna 'score_edad'
    df_edad['score_edad'] = df_edad['edad'].apply(puntaje_edad_nueva)

    # Calcular el Min-Max Scaling para la columna 'score_edad'
    min_valor = df_edad['score_edad'].min()
    max_valor = df_edad['score_edad'].max()
    rango_deseado = 100
    min_deseado = 0

    df_edad['score_edad_scaled'] = ((df_edad['score_edad'] - min_valor) / (max_valor - min_valor)) * rango_deseado + min_deseado

    # Multiplicar 'score_edad_scaled' por el ponderador
    df_edad['score_edad_weighted'] = df_edad['score_edad_scaled'] * edad_weight
    
    # Devolver el DataFrame con las columnas seleccionadas
    return df_edad[['dni', 'edad', 'score_edad', 'score_edad_scaled','score_edad_weighted']].sort_values(by='dni')

# aplicar funcion
df_edad = procesar_edad(df)
df_edad


,dni,edad,score_edad,score_edad_scaled,score_edad_weighted
55,10076758,75,30,74.358974,7.435897
1290,10171954,58,1,0.000000,0.000000
1192,10196231,79,30,74.358974,7.435897
608,10209961,77,30,74.358974,7.435897
348,10344303,73,30,74.358974,7.435897
...,...,...,...,...,...
1472,99794603,86,40,100.000000,10.000000
144,99913412,70,30,74.358974,7.435897
1286,99929175,76,30,74.358974,7.435897
24,99949389,72,30,74.358974,7.435897


### DISCAPACIDAD.SCORE

In [14]:
def puntaje_discapacidad(grado_dis):
    if grado_dis == 0:
        return 0
    elif grado_dis == 1:
        return 10
    else:  # grado_dis == 2
        return 20

def procesar_discapacidad(df):
    # Copiar el DataFrame para evitar modificar el original
    df_discapacidad = df.copy()

    # Aplicar la función a la columna 'edad' y crear la nueva columna 'score_edad'
    df_discapacidad['score_discapacidad'] = df_discapacidad['grado_dis'].apply(puntaje_discapacidad)

    # Calcular el Min-Max Scaling para la columna 'score_edad'
    min_valor = df_discapacidad['score_discapacidad'].min()
    max_valor = df_discapacidad['score_discapacidad'].max()
    rango_deseado = 100
    min_deseado = 0

    df_discapacidad['score_discapacidad_scaled'] = ((df_discapacidad['score_discapacidad'] - min_valor) / (max_valor - min_valor)) * rango_deseado + min_deseado

    df_discapacidad['score_discapacidad_weighted'] = df_discapacidad['score_discapacidad_scaled'] * discapacidad_weight
    
    # Devolver el DataFrame con las columnas seleccionadas
    return df_discapacidad[['dni', 'score_discapacidad', 'score_discapacidad_scaled','score_discapacidad_weighted']].sort_values(by='dni')

# aplicar funcion
df_discapacidad = procesar_discapacidad(df)
df_discapacidad

,dni,score_discapacidad,score_discapacidad_scaled,score_discapacidad_weighted
55,10076758,0,0.0,0.0
1290,10171954,0,0.0,0.0
1192,10196231,20,100.0,10.0
608,10209961,10,50.0,5.0
348,10344303,0,0.0,0.0
...,...,...,...,...
1472,99794603,20,100.0,10.0
144,99913412,0,0.0,0.0
1286,99929175,0,0.0,0.0
24,99949389,10,50.0,5.0


### SCORE FAMILIA NUMEROSA

In [18]:
def puntaje_familia(tipo_fam_num):
    if tipo_fam_num == 0:
        return 0
    elif tipo_fam_num == 1:
        return 5
    else:  # tipo_fam_num == 2
        return 10

def procesar_familia(df):
    # Copiar el DataFrame para evitar modificar el original
    df_familia = df.copy()

    # Aplicar la función a la columna 'edad' y crear la nueva columna 'score_edad'
    df_familia['score_familia'] = df_familia['tipo_fam_num'].apply(puntaje_familia)

    # Calcular el Min-Max Scaling para la columna 'score_edad'
    min_valor = df_discapacidad['score_familia'].min()
    max_valor = df_discapacidad['score_familia'].max()
    rango_deseado = 100
    min_deseado = 0

    df_familia['score_familia_scaled'] = ((df_familia['score_familia'] - min_valor) / (max_valor - min_valor)) * rango_deseado + min_deseado

    df_familia['score_familia_weighted'] = df_familia['score_familia_scaled'] * familia_weight
    
    # Devolver el DataFrame con las columnas seleccionadas
    return df_discapacidad[['dni', 'score_familia', 'score_familia_scaled','score_familia_weighted']].sort_values(by='dni')

# aplicar funcion
df_familia = procesar_discapacidad(df)
df_familia

,dni,score_discapacidad,score_discapacidad_scaled,score_discapacidad_weighted
55,10076758,0,0.0,0.0
1290,10171954,0,0.0,0.0
1192,10196231,20,100.0,10.0
608,10209961,10,50.0,5.0
348,10344303,0,0.0,0.0
...,...,...,...,...
1472,99794603,20,100.0,10.0
144,99913412,0,0.0,0.0
1286,99929175,0,0.0,0.0
24,99949389,10,50.0,5.0


### COCHE - ARRENDADOR - OFICIO ESPECIAL

In [34]:
def puntaje_coche_arrendador_oficio(coche_arrendador_oficio):
    if coche_arrendador_oficio == True:
        return 100
    else:  #  == False
        return 0


def procesar_coche_arrendador_oficio(df):
    # Copiar el DataFrame para evitar modificar el original
    df_coche_arrendador_oficio = df.copy()

    # Aplicar la función a la columna 'edad' y crear la nueva columna 'score_edad'
    df_coche_arrendador_oficio['score_coche'] = df_coche_arrendador_oficio['coche'].apply(puntaje_coche_arrendador_oficio)
    df_coche_arrendador_oficio['score_arrendador'] = df_coche_arrendador_oficio['arrendador'].apply(puntaje_coche_arrendador_oficio)
    df_coche_arrendador_oficio['score_oficio_especial'] = df_coche_arrendador_oficio['oficio_especial'].apply(puntaje_coche_arrendador_oficio)


    df_coche_arrendador_oficio['score_coche_weighted'] = df_coche_arrendador_oficio['score_coche'] * coche_wight
    df_coche_arrendador_oficio['score_arrendador_weighted'] = df_coche_arrendador_oficio['score_arrendador'] * arrendador_wight
    df_coche_arrendador_oficio['score_oficio_especial_weighted'] = df_coche_arrendador_oficio['score_oficio_especial'] * oficio_especial_wight
     
    # Devolver el DataFrame con las columnas seleccionadas
    return df_coche_arrendador_oficio[['dni', 'score_coche', 'score_coche_weighted','score_arrendador','score_arrendador_weighted','score_oficio_especial', 'score_oficio_especial_weighted' ]].sort_values(by='dni')


# aplicar funcion
df_coche_arrendador_oficio = procesar_coche_arrendador_oficio(df)
df_coche_arrendador_oficio

,dni,score_coche,score_coche_weighted,score_arrendador,score_arrendador_weighted,score_oficio_especial,score_oficio_especial_weighted
55,10076758,100,10.0,100,10.0,0,0.0
1290,10171954,100,10.0,100,10.0,100,10.0
1192,10196231,100,10.0,100,10.0,100,10.0
608,10209961,0,0.0,0,0.0,100,10.0
348,10344303,0,0.0,0,0.0,100,10.0
...,...,...,...,...,...,...,...
1472,99794603,0,0.0,100,10.0,100,10.0
144,99913412,100,10.0,100,10.0,0,0.0
1286,99929175,100,10.0,100,10.0,0,0.0
24,99949389,100,10.0,100,10.0,100,10.0
